<a href="https://colab.research.google.com/github/anastasiablokhina/Data_Visualization_with_Python_and_R/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

Выполнение пункта 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
sales_train = pd.read_csv('/content/drive/MyDrive/data/sales_train.csv')
items = pd.read_csv('/content/drive/MyDrive/data/items.csv')
with_items = pd.merge(sales_train, items, how='inner')
grouped_with_items = with_items.groupby(['item_id', 'item_name'])['item_cnt_day'].sum()
top_products = grouped_with_items.sort_values(ascending=False).head(10)
task_1 = pd.DataFrame(top_products)
task_1 = task_1.reset_index().set_index('item_id')
task_1

,item_name,item_cnt_day
item_id,,
20949,Corporate package T-shirt 1C Interest white (3...,187642.0
2808,"Diablo III [PC, Jewel, Russian version]",17245.0
3732,"Grand Theft Auto V [PS3, Russian subtitles]",16642.0
17717,Acceptance of funds for 1C-Online,15830.0
5822,Playstation Store replenishment of wallet: Pay...,14515.0
3734,"Grand Theft Auto V [Xbox 360, Russian subtitles]",11688.0
6675,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...,10289.0
3731,"Grand Theft Auto V [PC, Russian subtitles]",10099.0
1855,"Battlefield 4 [PC, Russian version]",10032.0


Выполнение пункта 2

In [ ]:
sales_train_indexed = sales_train.set_index('item_id')
only_task_1 = sales_train_indexed.loc[task_1.index.values]
only_task_1['year'] = only_task_1['date_block_num'].map(lambda date: date // 12 + 2013)
only_task_1 = only_task_1.groupby(['item_id', 'year'])
only_task_1 = only_task_1.size().unstack()
only_task_1

year,2013,2014,2015
item_id,,,
1855,1548.0,3535.0,598.0
2808,3924.0,2882.0,673.0
3731,NaN,NaN,2354.0
3732,2630.0,3041.0,804.0
3734,2305.0,2684.0,816.0
5822,2738.0,4107.0,2563.0
6675,685.0,3584.0,853.0
16787,2799.0,2157.0,327.0
17717,809.0,3890.0,4368.0


In [ ]:
#по таблице видно, что только один товар не продавался все три года, но можно еще раз в этом убедиться
only_task_1_2 = task_1.loc[only_task_1.dropna().index.values]
pd.DataFrame(only_task_1_2['item_name'])

,item_name
item_id,
1855,"Battlefield 4 [PC, Russian version]"
2808,"Diablo III [PC, Jewel, Russian version]"
3732,"Grand Theft Auto V [PS3, Russian subtitles]"
3734,"Grand Theft Auto V [Xbox 360, Russian subtitles]"
5822,Playstation Store replenishment of wallet: Pay...
6675,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...
16787,"Some of us [PS3, Russian version]"
17717,Acceptance of funds for 1C-Online
20949,Corporate package T-shirt 1C Interest white (3...


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

Выполнение пункта 1

In [ ]:
grouped = pd.DataFrame(sales_train.groupby('item_id')['item_cnt_day'].sum())
item_categories = pd.read_csv('/content/drive/MyDrive/data/item_categories.csv', index_col='item_category_id')
items_indexed = items.set_index('item_id')

for i in grouped.index.values:
    items_indexed.at[i, 'count_all'] = grouped.loc[i, 'item_cnt_day']
    
grouped_by_categories = pd.DataFrame(items_indexed.groupby('category_id')['count_all'].sum())
min_value = grouped_by_categories['count_all'].min()
id_min = grouped_by_categories['count_all'] == min_value
item_categories.loc[id_min]

,item_category_name
item_category_id,
10,Game consoles - PS2
51,Books - Cognitive literature


Выполнение пункта 2

In [ ]:
max_value = grouped_by_categories['count_all'].max()
id_max = grouped_by_categories['count_all'] == max_value
item_categories.loc[id_max]

,item_category_name
item_category_id,
40,Cinema - DVD


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


Выполнение пункта 1

In [ ]:
# пять магазинов
grouped_by_shops = pd.DataFrame(sales_train.groupby('shop_id')['item_cnt_day'].sum())
top_shops = grouped_by_shops.sort_values(by='item_cnt_day', ascending=False).head(5)

only_top_shops = sales_train.set_index('shop_id').loc[top_shops.index.values]

#товары из категорий 10 и 51
items_from_2_1 = items.set_index('category_id').loc[id_min]

#товары из категории 40
items_from_2_2 = items.set_index('category_id').loc[id_max]

In [ ]:
grouped_only_top_shops = pd.DataFrame(only_top_shops.groupby('item_id')['item_cnt_day'].mean()).reset_index()
only_items_from_2_1 = pd.merge(items_from_2_1, grouped_only_top_shops, how='inner')
only_items_from_2_1 #категории 10,51 не продавались в магазинах с наивысшими продажами, поэтому вывод пустой

,item_name,item_id,item_cnt_day


In [ ]:
only_items_from_2_2 = pd.merge(items_from_2_2, grouped_only_top_shops, how='inner')
only_items_from_2_2

,item_name,item_id,item_cnt_day
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,1.000000
1,*** IN THE GLORY OF THE GLORY (UNV) D,2,1.000000
2,*** BLUE WAVE (Univ) D,3,1.000000
3,*** BOX (GLASS) D,4,1.000000
4,*** NEW AMERICAN GRAPHICS (UNI) D,5,1.000000
...,...,...,...
4709,YALTA 45,22157,1.000000
4710,YANKOVSKY,22159,1.142857
4711,FAIR OF FAIRY (Region),22160,1.000000
4712,THE FATHER,22162,1.481283


Выполнение пункта 2

In [ ]:
grouped_only_top_shops_min = pd.DataFrame(only_top_shops.groupby(['item_id', 'date_block_num'])['item_cnt_day'].min()).reset_index()
only_items_from_2_1_min = pd.merge(items_from_2_1, grouped_only_top_shops_min, how='inner')
only_items_from_2_1_min #категории 10,51 не продавались в магазинах с наивысшими продажами, поэтому вывод пустой

,item_name,item_id,date_block_num,item_cnt_day


In [ ]:
grouped_only_top_shops_max = pd.DataFrame(only_top_shops.groupby(['item_id', 'date_block_num'])['item_cnt_day'].max()).reset_index()
only_items_from_2_1_max = pd.merge(items_from_2_1, grouped_only_top_shops_max, how='inner')
only_items_from_2_1_max #категории 10,51 не продавались в магазинах с наивысшими продажами, поэтому вывод пустой

,item_name,item_id,date_block_num,item_cnt_day


In [ ]:
only_items_from_2_2_min = pd.merge(items_from_2_2, grouped_only_top_shops_min, how='inner')
only_items_from_2_2_min.drop_duplicates(subset = 'item_id').set_index('item_id')

,item_name,date_block_num,item_cnt_day
item_id,,,
0,!! IN THE POWER OF HAPPINESS (PLAST) D,20,1.0
2,*** IN THE GLORY OF THE GLORY (UNV) D,19,1.0
3,*** BLUE WAVE (Univ) D,18,1.0
4,*** BOX (GLASS) D,20,1.0
5,*** NEW AMERICAN GRAPHICS (UNI) D,23,1.0
...,...,...,...
22157,YALTA 45,4,1.0
22159,YANKOVSKY,28,1.0
22160,FAIR OF FAIRY (Region),0,1.0


In [ ]:
only_items_from_2_2_max = pd.merge(items_from_2_2, grouped_only_top_shops_max, how='inner')
only_items_from_2_2_max.drop_duplicates(subset = 'item_id').set_index('item_id')

,item_name,date_block_num,item_cnt_day
item_id,,,
0,!! IN THE POWER OF HAPPINESS (PLAST) D,20,1.0
2,*** IN THE GLORY OF THE GLORY (UNV) D,19,1.0
3,*** BLUE WAVE (Univ) D,18,1.0
4,*** BOX (GLASS) D,20,1.0
5,*** NEW AMERICAN GRAPHICS (UNI) D,23,1.0
...,...,...,...
22157,YALTA 45,4,1.0
22159,YANKOVSKY,28,2.0
22160,FAIR OF FAIRY (Region),0,1.0


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

Выполнение пункта 1

In [ ]:
#для категории 10 (с наименьшими продажами)
items_10 = items.set_index('category_id').loc[[10]]
only_items_10 = pd.merge(items_10, sales_train, how='inner')
df1 = pd.DataFrame(only_items_10.groupby('date_block_num', as_index = False)['item_price'].mean())
df1.rename(columns = {'item_price' : 'item_mean_price'}, inplace = True)
df2 = pd.DataFrame(only_items_10.groupby('date_block_num', as_index = False)['item_price'].median())
df2.rename(columns = {'item_price' : 'item_median_price'}, inplace = True)
task_3_1_1 = pd.merge(df1, df2).set_index('date_block_num')
task_3_1_1

,item_mean_price,item_median_price
date_block_num,,
3,1998.0,1998.0


In [ ]:
#для категории 51 (с наименьшими продажами)
items_51 = items.set_index('category_id').loc[[51]]
only_items_51 = pd.merge(items_51, sales_train, how='inner')
df3 = pd.DataFrame(only_items_51.groupby('date_block_num', as_index = False)['item_price'].mean())
df3.rename(columns = {'item_price' : 'item_mean_price'}, inplace = True)
df4 = pd.DataFrame(only_items_51.groupby('date_block_num', as_index = False)['item_price'].median())
df4.rename(columns = {'item_price' : 'item_median_price'}, inplace = True)
task_3_1_2 = pd.merge(df3, df4).set_index('date_block_num')
task_3_1_2

,item_mean_price,item_median_price
date_block_num,,
18,129.0,129.0


In [ ]:
#для категории 40 (с наивысшими продажами)
items_40 = items.set_index('category_id').loc[40] 
only_items_40 = pd.merge(items_40, sales_train, how='inner')
df5 = pd.DataFrame(only_items_40.groupby('date_block_num', as_index = False)['item_price'].mean())
df5.rename(columns = {'item_price' : 'item_mean_price'}, inplace = True)
df6 = pd.DataFrame(only_items_40.groupby('date_block_num', as_index = False)['item_price'].median())
df6.rename(columns = {'item_price' : 'item_median_price'}, inplace = True)
task_3_1_3 = pd.merge(df5, df6).set_index('date_block_num')
task_3_1_3

,item_mean_price,item_median_price
date_block_num,,
0,244.485942,199.00
1,252.174936,203.00
2,260.645016,231.00
3,254.852406,199.00
4,247.627583,199.00
5,245.860855,199.00
6,236.812052,149.00
7,234.470179,149.00
8,241.818554,198.00


Выполнение пункта 2

In [ ]:
#для категории 10 (с наименьшими продажами)
arr_2013, arr_2014, arr_2015 = [], [], []
for i in task_3_1_1.index.values[:-2]:
  if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i, 'item_mean_price']:
    if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i + 2, 'item_mean_price']:
      if i + 1 < 12:
        arr_2013.append(i+1)
      elif i + 1 < 24:
        arr_2014.append(i+1)
      else:
        arr_2015.append(i+1)
print(' 2013:', arr_2013, '\n', '2014:', arr_2014, '\n', '2015:',  arr_2015)
#вывод пустой, тк в категории 10 только 1 товар

 2013: [] 
 2014: [] 
 2015: []


In [ ]:
#для категории 51 (с наименьшими продажами)
arr_2013, arr_2014, arr_2015 = [], [], []
for i in task_3_1_2.index.values[:-2]:
  if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i, 'item_mean_price']:
    if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i + 2, 'item_mean_price']:
      if i + 1 < 12:
        arr_2013.append(i+1)
      elif i + 1 < 24:
        arr_2014.append(i+1)
      else:
        arr_2015.append(i+1)
print(' 2013:', arr_2013, '\n', '2014:', arr_2014, '\n', '2015:',  arr_2015)
#вывод пустой, тк в категории 51 только 1 товар

 2013: [] 
 2014: [] 
 2015: []


In [ ]:
#для категории 40 (с наивысшими продажами)
arr_2013, arr_2014, arr_2015 = [], [], []
for i in task_3_1_3.index.values[:-2]:
  if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i, 'item_mean_price']:
    if task_3_1_3.at[i + 1, 'item_mean_price'] < task_3_1_3.at[i + 2, 'item_mean_price']:
      if i + 1 < 12:
        arr_2013.append(i+1)
      elif i + 1 < 24:
        arr_2014.append(i+1)
      else:
        arr_2015.append(i+1)
print(' 2013:', arr_2013, '\n', '2014:', arr_2014, '\n', '2015:',  arr_2015)


 2013: [7] 
 2014: [12, 16, 21, 23] 
 2015: [25, 31]
